In [0]:
# Second iteration of this function to build up the dataframe

import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius as genius
import pandas as pd
import json
import time

# #################################################
#   Spotify API user credentials, generate token
CLIENT_ID = <CLIENT_ID>
CLIENT_SECRET = <CLIENT_SECRET>
credentials = SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

token = credentials.get_access_token(as_dict=False)
spotify = spotipy.Spotify(auth=token)
spotify.trace = False
# #################################################

# #################################################
#   Genius API settings and user credentials
genius_api = genius.Genius(<GENIUS_CREDENTIALS>)
genius.verbose = True                         # Keep status messages on
genius.remove_section_headers = False         # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title
# #################################################

cols =['track_name',
       'artist_name',
       'track_id',
       'song_spotify_link',
       'artist_id',
       'release_date',
       'release_date_precision',
       'duration',
       'popularity',
       'explicit',
       'audio_preview',
       'cover_art',
       'danceability',
       'energy',
       'key',
       'loudness',
       'mode',
       'speechiness',
       'acousticness',
       'instrumentalness',
       'liveness',
       'valence',
       'tempo',
       'time_sig',
       'num_artist_followers',
       'artist_genres',
       'artist_popularity',
       'artist_spotify_avatar',
       'artist_spotify_link']

df = pd.DataFrame(columns=cols)

genre_list = pd.read_csv('max_genres.csv')

genre_list = genre_list[1368:]

start = time.time()

# #################################################
#      Initial user query for a track name
# #################################################

#results = spotify.search(q='track:' + "Thriller", type='track', limit=1)
#results = spotify.search("Californication", type="track", limit=1)

for genre in genre_list['genre']:
  print(genre)
  api_query = 'genre:' + genre
  results = spotify.search(q=api_query, limit=50, market='US')
  time.sleep(1)
  data = {}
  if len(results['tracks']['items']) > 0:
    for i in range(0,len(results['tracks']['items'])):
      for j in cols:
        data[j] = []
      # API calls
      track = results['tracks']['items'][i]
      audio_features = spotify.audio_features(str(track['uri']))[0]
      primary_artist = spotify.artist(str(track['album']['artists'][0]['uri']))
      # Build features
      data['track_name'] = track['name']
      data['artist_name'] = track['artists'][0]['name']
      data['track_id'] = track['id']
      data['song_spotify_link'] = track['external_urls']['spotify']
      data['artist_id'] = track['artists'][0]['id']
      data['release_date'] = track['album']['release_date']
      data['release_date_precision'] = track['album']['release_date_precision']
      data['duration'] = track['duration_ms']
      data['popularity'] = track['popularity']
      data['explicit'] = track['explicit']
      data['audio_preview'] = track['preview_url']
      data['cover_art'] = track['album']['images'][1]['url']
      data['danceability'] = audio_features['danceability']
      data['energy'] = audio_features['energy']
      data['key'] = audio_features['key']
      data['loudness'] = audio_features['loudness']
      data['mode'] = audio_features['mode']
      data['speechiness'] = audio_features['speechiness']
      data['acousticness'] = audio_features['acousticness']
      data['instrumentalness'] = audio_features['instrumentalness']
      data['liveness'] = audio_features['liveness']
      data['valence'] = audio_features['valence']
      data['tempo'] = audio_features['tempo']
      data['time_sig'] = audio_features['time_signature']
      data['num_artist_followers'] = primary_artist['followers']['total']
      data['artist_genres'] = primary_artist['genres']
      data['artist_popularity'] = primary_artist['popularity']
      data['artist_spotify_avatar'] = primary_artist['images'][1]['url']
      data['artist_spotify_link'] = primary_artist['external_urls']['spotify']
      df = df.append(data, ignore_index=True)
      time.sleep(0.2)
  else:
    print('Track not found')
  time.sleep(0.5)